In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('/home/kazuki_onodera/Python')
import lgbmextension as ex
import lightgbm as lgb
import multiprocessing
from gplearn.genetic import SymbolicTransformer
import gc
import EDA
import utils

In [2]:
SEED = 71

In [3]:
#X = utils.load_train().drop(['SK_ID_CURR'], axis=1)
X = utils.read_pickles('../data/101_train')
y = utils.read_pickles('../data/label').TARGET
X.shape

100%|██████████| 20/20 [00:00<00:00, 35.06it/s]


(307511, 120)

In [8]:
param = {
         'objective': 'binary',
         'metric': 'auc',
         'learning_rate': 0.05,
         'max_depth': -1,
         'num_leaves': 255,
         'max_bin': 255,
         'colsample_bytree': 0.5,
         'subsample': 0.5,
         'nthread': multiprocessing.cpu_count(),
         'bagging_freq': 1,
         
         'seed': SEED
         }


categorical_feature = ['NAME_CONTRACT_TYPE',
                     'CODE_GENDER',
                     'FLAG_OWN_CAR',
                     'FLAG_OWN_REALTY',
                     'NAME_TYPE_SUITE',
                     'NAME_INCOME_TYPE',
                     'NAME_EDUCATION_TYPE',
                     'NAME_FAMILY_STATUS',
                     'NAME_HOUSING_TYPE',
                     'OCCUPATION_TYPE',
                     'WEEKDAY_APPR_PROCESS_START',
                     'ORGANIZATION_TYPE',
                     'FONDKAPREMONT_MODE',
                     'HOUSETYPE_MODE',
                     'WALLSMATERIAL_MODE',
                     'EMERGENCYSTATE_MODE']


# CV ( benchmark )

In [11]:
dtrain = lgb.Dataset(X, y, categorical_feature=categorical_feature)
ret = lgb.cv(param, dtrain, 9999, nfold=5,
             early_stopping_rounds=50, verbose_eval=10,
             seed=SEED)
ret['auc-mean'][-1]

[10]	cv_agg's auc: 0.729411 + 0.00215306
[20]	cv_agg's auc: 0.736183 + 0.00217633
[30]	cv_agg's auc: 0.741978 + 0.00206874
[40]	cv_agg's auc: 0.743821 + 0.0023523
[50]	cv_agg's auc: 0.746626 + 0.00228584
[60]	cv_agg's auc: 0.747173 + 0.00228664
[70]	cv_agg's auc: 0.747679 + 0.0020249
[80]	cv_agg's auc: 0.748786 + 0.00179863
[90]	cv_agg's auc: 0.749196 + 0.00171387
[100]	cv_agg's auc: 0.749337 + 0.00141182
[110]	cv_agg's auc: 0.74968 + 0.00179155
[120]	cv_agg's auc: 0.749964 + 0.00172182
[130]	cv_agg's auc: 0.750274 + 0.00157808
[140]	cv_agg's auc: 0.750476 + 0.00151443
[150]	cv_agg's auc: 0.750359 + 0.0015564
[160]	cv_agg's auc: 0.750387 + 0.00163539
[170]	cv_agg's auc: 0.750379 + 0.00169016
[180]	cv_agg's auc: 0.750083 + 0.00169213
[190]	cv_agg's auc: 0.749825 + 0.00167055


0.7505534358850445

# CV ( encoding )

In [17]:
X = utils.load_train().drop(['SK_ID_CURR'], axis=1)

100%|██████████| 20/20 [00:03<00:00,  6.21it/s]


In [29]:
tmp = X.drop('TARGET', axis=1)

In [35]:
enc = ce.LeaveOneOutEncoder(cols=categorical_feature)

In [36]:
tmp_ = enc.fit_transform(tmp, y)

In [37]:
EDA.df_info(tmp_[categorical_feature])

Shape: (307511, 16)


,DataType,#Nulls,#Uniques,Min,Mean,Max,Std,top10 val,top10 cnt
NAME_CONTRACT_TYPE,float64,0,2,0.054783,0.080729,0.083459,0.008417,"[0.08345912763449208, 0.05478329177909082]","[278232, 29279]"
CODE_GENDER,float64,0,3,0.000000,0.080729,0.101419,0.014907,"[0.06999328222555916, 0.1014192025433328, 0.0]","[202448, 105059, 4]"
FLAG_OWN_CAR,float64,0,2,0.072437,0.080729,0.085002,0.005953,"[0.08500226685852831, 0.07243730100299273]","[202924, 104587]"
FLAG_OWN_REALTY,float64,0,2,0.079616,0.080729,0.083249,0.001675,"[0.07961577407740775, 0.08324929139375152]","[213312, 94199]"
NAME_TYPE_SUITE,float64,0,8,0.073768,0.080840,0.098305,0.002816,"[0.08183047246565751, 0.07494582679518792, 0.0...","[248526, 40149, 11370, 3267, 1770, 1292, 866, ..."
NAME_INCOME_TYPE,float64,0,7,0.000000,0.080729,0.400000,0.017393,"[0.09588471664126369, 0.07484256531270508, 0.0...","[158774, 71617, 55362, 21703, 28, 22, 5]"
NAME_EDUCATION_TYPE,float64,0,5,0.018293,0.080729,0.109277,0.015683,"[0.08939928843221562, 0.05355115344028425, 0.0...","[218391, 74863, 10277, 3816, 164]"
NAME_FAMILY_STATUS,float64,0,6,0.000000,0.080729,0.099446,0.011036,"[0.07559868045939562, 0.0980767538068832, 0.09...","[196432, 45444, 29775, 19770, 16088, 2]"
NAME_HOUSING_TYPE,float64,0,6,0.065724,0.080729,0.123131,0.010074,"[0.07795710746588094, 0.1169811320754717, 0.08...","[272868, 14840, 11183, 4881, 2617, 1122]"
OCCUPATION_TYPE,float64,0,19,0.048303,0.085618,0.171524,0.019403,"[0.08072881945686496, 0.10578769977892943, 0.0...","[96391, 55186, 32102, 27570, 21371, 18603, 113..."


In [38]:
tmp_.groupby('ORGANIZATION_TYPE').TARGET.mean()

ORGANIZATION_TYPE
Advertising               0.081585
Agriculture               0.104727
Bank                      0.051855
Business Entity Type 1    0.081384
Business Entity Type 2    0.085284
Business Entity Type 3    0.092996
Cleaning                  0.111538
Construction              0.116798
Culture                   0.055409
Electricity               0.066316
Emergency                 0.071429
Government                0.069781
Hotel                     0.064182
Housing                   0.079446
Industry: type 1          0.110683
Industry: type 10         0.064220
Industry: type 11         0.086538
Industry: type 12         0.037940
Industry: type 13         0.134328
Industry: type 2          0.072052
Industry: type 3          0.106162
Industry: type 4          0.101482
Industry: type 5          0.068447
Industry: type 6          0.071429
Industry: type 7          0.080337
Industry: type 8          0.125000
Industry: type 9          0.066805
Insurance                 0.056951
Ki

In [42]:
dtrain = lgb.Dataset(tmp_, y, 
#                     categorical_feature=categorical_feature
                    )
ret = lgb.cv(param, dtrain, 9999, nfold=5,
             early_stopping_rounds=50, verbose_eval=10,
             seed=SEED)

ValueError: DataFrame.dtypes for data must be int, float or bool. Did not expect the data types in fields NAME_CONTRACT_TYPE, CODE_GENDER, FLAG_OWN_CAR, FLAG_OWN_REALTY, NAME_TYPE_SUITE, NAME_INCOME_TYPE, NAME_EDUCATION_TYPE, NAME_FAMILY_STATUS, NAME_HOUSING_TYPE, OCCUPATION_TYPE, WEEKDAY_APPR_PROCESS_START, ORGANIZATION_TYPE, FONDKAPREMONT_MODE, HOUSETYPE_MODE, WALLSMATERIAL_MODE, EMERGENCYSTATE_MODE

# prev

In [44]:
X = utils.load_train().set_index('SK_ID_CURR')

100%|██████████| 20/20 [00:00<00:00, 45.36it/s]


In [3]:
prev = utils.read_pickles('../data/previous_application')

100%|██████████| 20/20 [00:04<00:00,  4.84it/s]


In [46]:
prev = prev.sort_values(['SK_ID_CURR', 'DAYS_LAST_DUE_1ST_VERSION'], ascending=[True, False]).drop_duplicates('SK_ID_CURR', keep='first')

In [4]:
prev.groupby('SK_ID_CURR').size().value_counts()

1     68952
2     42169
3     26905
4     18496
5     12364
6      8587
7      6262
8      4449
9      3161
10     2408
11     1777
12     1372
13     1052
14      746
15      631
16      466
17      360
18      313
19      231
20      176
21      159
22      122
23       98
24       96
25       71
26       60
27       57
28       33
29       27
30       24
31       19
33       14
32       13
34       13
36        9
37        9
35        8
39        7
38        6
41        4
43        4
44        4
52        3
60        2
46        2
54        2
42        2
55        2
62        1
58        1
56        1
65        1
67        1
45        1
51        1
49        1
48        1
47        1
40        1
70        1
dtype: int64

In [49]:
prev = prev.drop('SK_ID_PREV', axis=1).set_index('SK_ID_CURR')

In [50]:
tmp = pd.concat([X, prev.add_prefix('prev_')], axis=1)

In [53]:
cat = [c for c in tmp.columns if tmp[c].dtype=='O']

In [54]:

from sklearn.preprocessing import LabelEncoder

In [55]:

le = LabelEncoder()
for c in cat:
    tmp[c].fillna('na dayo', inplace=True)
    tmp[c] = le.fit_transform(tmp[c])

In [56]:
y = tmp.TARGET
tmp.drop('TARGET', axis=1, inplace=True)

In [ ]:
dtrain = lgb.Dataset(tmp.iloc[0:,0:20], y, 
                     #categorical_feature=cat
                    )
ret = lgb.cv(param, dtrain, 9999, nfold=5,
             early_stopping_rounds=50, verbose_eval=10,
             seed=SEED)

In [67]:
cat

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE',
 'prev_NAME_CONTRACT_TYPE',
 'prev_WEEKDAY_APPR_PROCESS_START',
 'prev_NAME_CASH_LOAN_PURPOSE',
 'prev_NAME_CONTRACT_STATUS',
 'prev_NAME_PAYMENT_TYPE',
 'prev_CODE_REJECT_REASON',
 'prev_NAME_TYPE_SUITE',
 'prev_NAME_CLIENT_TYPE',
 'prev_NAME_GOODS_CATEGORY',
 'prev_NAME_PORTFOLIO',
 'prev_NAME_PRODUCT_TYPE',
 'prev_CHANNEL_TYPE',
 'prev_NAME_SELLER_INDUSTRY',
 'prev_NAME_YIELD_GROUP',
 'prev_PRODUCT_COMBINATION']

In [65]:
tmp.max().max()

117000000.0

In [1]:
prev

NameError: name 'prev' is not defined